# Część 1: Przygotowanie Danych i Inżynieria Cech
Ta część jest odpowiedzialna za wczytanie, oczyszczenie i przetworzenie danych w celu przygotowania ich do treningu modelu. Zapiszemy również wszystkie niezbędne artefakty (mapy, transformatory) na dysku.

In [1]:
import pandas as pd
import numpy as np
import os
import re
import joblib
import gc
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

# Definicja stałych globalnych
ARTIFACTS_DIR = 'artifacts'
PROCESSED_DATA_DIR = 'processed_data'
MAX_TEXT_FEATURES = 10000
RANDOM_STATE = 42

# Tworzenie katalogów, jeśli nie istnieją
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"Katalogi '{ARTIFACTS_DIR}' i '{PROCESSED_DATA_DIR}' są gotowe.")

Katalogi 'artifacts' i 'processed_data' są gotowe.


## Wczytywanie Danych
W tej komórce wczytujemy wszystkie niezbędne pliki CSV: dane o ofertach, definicję hierarchii lokalizacji oraz mapę do standaryzacji nazw.

In [5]:
# KOMÓRKA 2 (Kod) - POPRAWIONA, Z WŁAŚCIWYM SEPARATOREM

def load_location_data(location_path='lokalizacja.csv'):
    """Wczytuje dane o hierarchii lokalizacji."""
    print(f"Wczytywanie pliku: {location_path}")
    # KLUCZOWA ZMIANA: Wracamy do przecinka, bo dane są nim rozdzielone.
    # Dodajemy nazwy kolumn, bo plik ich nie ma.
    df_loc = pd.read_csv(
        location_path,
        na_values=['\\N', 'NULL'],
        sep=',',
        header=None,
        names=['id', 'parent_id', 'name', 'type', 'full_name'] # Zgodnie z promptem + dodatkowe
    )
    return df_loc

def load_offers_data(offers_path='saleflats_2024_2025.csv'):
    """Wczytuje dane o ofertach, dynamicznie znajdując ostatnią kolumnę."""
    print(f"Wczytywanie pliku: {offers_path}")
    column_names = ['title', 'description', 'area', 'price', 'locationPath']
    
    try:
        # KLUCZOWA ZMIANA: Wracamy do przecinka
        first_row = pd.read_csv(offers_path, header=None, sep=',', nrows=1, on_bad_lines='skip')
        last_col_index = first_row.shape[1] - 1
        print(f"Wykryto {last_col_index + 1} kolumn. Ostatnia kolumna ma indeks: {last_col_index}")
        
        usecols = [3, 4, 5, 6, last_col_index]
        
        df_offers = pd.read_csv(offers_path, header=None, sep=',', usecols=usecols, on_bad_lines='skip')
        df_offers.columns = column_names
        
    except Exception as e:
        print(f"Wystąpił błąd: {e}. Upewnij się, że plik '{offers_path}' ma co najmniej 7 kolumn i używa przecinka jako separatora.")
        df_offers = pd.DataFrame(columns=column_names)
        
    return df_offers

# Wywołanie funkcji
df_loc = load_location_data()
df_offers = load_offers_data()

# Wyświetlenie informacji o wczytanych danych
print("\n--- Informacje o danych lokalizacyjnych (df_loc) ---")
df_loc.info()
print("\n--- Nagłówek danych lokalizacyjnych ---")
print(df_loc.head())

print("\n--- Informacje o danych ofert (df_offers) ---")
df_offers.info()
print("\n--- Nagłówek danych ofert ---")
print(df_offers.head())

Wczytywanie pliku: lokalizacja.csv
Wczytywanie pliku: saleflats_2024_2025.csv
Wykryto 53 kolumn. Ostatnia kolumna ma indeks: 52

--- Informacje o danych lokalizacyjnych (df_loc) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398832 entries, 0 to 398831
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         398832 non-null  int64  
 1   parent_id  398816 non-null  float64
 2   name       398832 non-null  object 
 3   type       398832 non-null  object 
 4   full_name  398832 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 15.2+ MB

--- Nagłówek danych lokalizacyjnych ---
   id  parent_id                name         type           full_name
0   1        NaN      Świętokrzyskie  Województwo      Świętokrzyskie
1   2        NaN             Śląskie  Województwo             Śląskie
2   3        NaN           Podlaskie  Województwo           Podlaskie
3   4        NaN            Opolskie 

## Budowa Hierarchii i Standaryzacja
Tworzymy kluczowe mapy (`id_to_name`, `hierarchy_map`) na podstawie danych o lokalizacjach. Standaryzujemy również nazwy w obu ramkach danych, aby zapewnić spójność.

In [6]:
# KOMÓRKA 3 (Kod) - POPRAWIONA

# 1. Stworzenie i zapisanie mapy id -> nazwa
# Upewniamy się, że kolumny są poprawnie nazwane (zgodnie z promptem)
id_to_name = dict(zip(df_loc['id'], df_loc['name']))
joblib.dump(id_to_name, os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))
print(f"Mapa 'id_to_name' została stworzona i zapisana. Liczba wpisów: {len(id_to_name)}")

# 2. Stworzenie i zapisanie mapy hierarchii
hierarchy_map = dict(zip(df_loc['id'], df_loc['parent_id']))
joblib.dump(hierarchy_map, os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
print(f"Mapa 'hierarchy_map' została stworzona i zapisana. Liczba wpisów: {len(hierarchy_map)}")

print("\n--- Przykładowe ścieżki (bez zmian, bo to już ID) ---")
print(df_offers[['locationPath']].head())

Mapa 'id_to_name' została stworzona i zapisana. Liczba wpisów: 398832
Mapa 'hierarchy_map' została stworzona i zapisana. Liczba wpisów: 398832

--- Przykładowe ścieżki (bez zmian, bo to już ID) ---
               locationPath
0      3,0,0,352,0,103786,0
1       3,0,0,352,0,99764,0
2       3,0,0,352,0,74375,0
3  3,0,0,352,0,74375,517513
4       3,0,0,352,0,95559,0


## Inżynieria Cech i Definicja Zmiennej Celu
W tej sekcji tworzymy zmienną celu (`target_location_id`) oraz cechy, które posłużą do treningu modelu: połączone cechy tekstowe, cechy numeryczne oraz cechę kategoryczną (miasto).

In [7]:
# KOMÓRKA 4 (Kod) - CAŁKOWICIE NOWA LOGIKA

# --- Zmienna Celu (Target) ---
# Funkcja do parsowania ścieżki ID i znajdowania najbardziej szczegółowego ID
def get_most_specific_id(path_str):
    if not isinstance(path_str, str):
        return np.nan
    # Dzielimy ścieżkę i konwertujemy na liczby, ignorując błędy
    ids = pd.to_numeric(path_str.split(','), errors='coerce')
    # Znajdujemy ostatnią wartość, która nie jest NaN i nie jest zerem
    for part_id in reversed(ids):
        if pd.notna(part_id) and part_id != 0:
            return int(part_id)
    return np.nan

# Funkcja do wyciągania ID miasta (zgodnie z logiką v6, miasto to 4 element, indeks 3)
def get_city_id_from_path(path_str):
    if not isinstance(path_str, str):
        return np.nan
    ids = path_str.split(',')
    if len(ids) > 3: # Musi być co najmniej 4 elementy
        city_id = pd.to_numeric(ids[3], errors='coerce')
        return city_id if city_id != 0 else np.nan
    return np.nan

# Stworzenie kolumny `target_location_id`
df_offers['target_location_id'] = df_offers['locationPath'].apply(get_most_specific_id)

# Stworzenie kolumny `city_id`
df_offers['city_id'] = df_offers['locationPath'].apply(get_city_id_from_path)

# Usunięcie ofert bez znalezionego ID celu lub miasta
initial_rows = len(df_offers)
df_offers.dropna(subset=['target_location_id', 'city_id'], inplace=True)
print(f"Usunięto {initial_rows - len(df_offers)} ofert bez poprawnego ID celu lub miasta.")

# Konwersja ID na typ integer
df_offers['target_location_id'] = df_offers['target_location_id'].astype(int)
df_offers['city_id'] = df_offers['city_id'].astype(int)

# --- Cechy Tekstowe ---
df_offers['text_features'] = df_offers['title'].fillna('') + " " + df_offers['description'].fillna('')

# --- Cechy Numeryczne ---
df_offers['area'] = pd.to_numeric(df_offers['area'], errors='coerce')
df_offers['price'] = pd.to_numeric(df_offers['price'], errors='coerce')

# --- Utworzenie finalnego DataFrame'u ---
df_processed = df_offers[['text_features', 'area', 'price', 'city_id', 'target_location_id']].copy()
# Dodatkowe czyszczenie na wypadek, gdyby konwersja numeryczna zawiodła
df_processed.dropna(inplace=True)

print("\n--- Informacje o finalnym, przetworzonym DataFrame (df_processed) ---")
df_processed.info()
print("\n--- Nagłówek df_processed ---")
print(df_processed.head())

Usunięto 134725 ofert bez poprawnego ID celu lub miasta.

--- Informacje o finalnym, przetworzonym DataFrame (df_processed) ---
<class 'pandas.core.frame.DataFrame'>
Index: 1112222 entries, 0 to 1305523
Data columns (total 5 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   text_features       1112222 non-null  object 
 1   area                1112222 non-null  float64
 2   price               1112222 non-null  float64
 3   city_id             1112222 non-null  int32  
 4   target_location_id  1112222 non-null  int32  
dtypes: float64(2), int32(2), object(1)
memory usage: 42.4+ MB

--- Nagłówek df_processed ---
                                       text_features   area     price  \
0  Mieszkanie trzypokojowe na sprzedaż Mieszkanie...  73.00  766500.0   
1  Sprzedam mieszkanie na parterze 64.8m2 Białyst...  64.80  540000.0   
2  Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki ...  51.00  540000.0   
3  Mieszkanie trzypoko

## Transformacja Cech i Podział na Zbiory
Dzielimy dane na zbiory treningowy i walidacyjny. Następnie dopasowujemy transformatory (imputer, scaler, vectorizer) na zbiorze treningowym i zapisujemy je, aby móc ich użyć w przyszłości.

In [9]:
# KOMÓRKA 5 (Kod) - POPRAWIONA WERSJA

# Podział na zbiory
X = df_processed.drop('target_location_id', axis=1)
y = df_processed['target_location_id']

# --- KLUCZOWA ZMIANA: Filtracja klas z jednym członkiem ---
print("Sprawdzanie liczności klas dla stratyfikacji...")
# 1. Policz, ile razy występuje każde miasto
city_counts = X['city_id'].value_counts()
# 2. Zidentyfikuj miasta, które mają więcej niż jedno ogłoszenie
valid_cities = city_counts[city_counts > 1].index
# 3. Przefiltruj dane, zostawiając tylko wiersze z "ważnymi" miastami
X_filtered = X[X['city_id'].isin(valid_cities)]
y_filtered = y[X['city_id'].isin(valid_cities)]
print(f"Usunięto {len(X) - len(X_filtered)} wierszy należących do miast z tylko jednym ogłoszeniem.")
# --- KONIEC ZMIANY ---

# Teraz używamy przefiltrowanych danych do podziału
X_train, X_val, y_train, y_val = train_test_split(
    X_filtered, 
    y_filtered, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=X_filtered['city_id'] # Stratyfikacja na przefiltrowanych danych
)

print(f"Podział danych: {len(X_train)} próbek treningowych, {len(X_val)} próbek walidacyjnych.")

# --- Przetwarzanie Cech Numerycznych ---
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

X_train_num_imputed = imputer.fit_transform(X_train[['area', 'price']])
X_train_num = scaler.fit_transform(X_train_num_imputed)

joblib.dump(imputer, os.path.join(ARTIFACTS_DIR, 'imputer.joblib'))
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))
print("Imputer i Scaler zostały dopasowane i zapisane.")

X_val_num_imputed = imputer.transform(X_val[['area', 'price']])
X_val_num = scaler.transform(X_val_num_imputed)

# --- Przetwarzanie Cech Tekstowych ---
vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES, ngram_range=(1, 2))
X_train_text = vectorizer.fit_transform(X_train['text_features'])

joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))
print("TfidfVectorizer został dopasowany i zapisany.")

X_val_text = vectorizer.transform(X_val['text_features'])

# --- Przygotowanie pozostałych cech ---
X_train_city = X_train['city_id'].values
X_val_city = X_val['city_id'].values

Sprawdzanie liczności klas dla stratyfikacji...
Usunięto 2757 wierszy należących do miast z tylko jednym ogłoszeniem.
Podział danych: 887572 próbek treningowych, 221893 próbek walidacyjnych.
Imputer i Scaler zostały dopasowane i zapisane.
TfidfVectorizer został dopasowany i zapisany.


## Zapisanie Wyników do Plików
Wszystkie przetworzone dane oraz kluczowe ramki danych są zapisywane na dysku. Umożliwi to ich wczytanie w drugiej części notebooka bez potrzeby ponownego przetwarzania.

In [10]:
# Zapis macierzy TF-IDF (jako sparse)
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'), X_train_text)
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'), X_val_text)

# Zapis macierzy numerycznych
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'), X_train_num)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'), X_val_num)

# Zapis ID miast
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'), X_train_city)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'), X_val_city)

# Zapis zmiennych docelowych
y_train.to_csv(os.path.join(PROCESSED_DATA_DIR, 'y_train.csv'), index=False, header=True)
y_val.to_csv(os.path.join(PROCESSED_DATA_DIR, 'y_val.csv'), index=False, header=True)

# Zapis ramek danych na potrzeby analizy
df_offers.to_csv(os.path.join(PROCESSED_DATA_DIR, 'df_offers_original_with_ids.csv'), index=False)
df_processed.to_csv(os.path.join(PROCESSED_DATA_DIR, 'df_processed.csv'), index=False)
print("Wszystkie przetworzone zbiory danych zostały zapisane.")

# Zwolnienie pamięci
del df_loc, df_offers, df_processed, X, y, X_train, X_val, y_train, y_val
del X_train_text, X_val_text, X_train_num, X_val_num, X_train_city, X_val_city
gc.collect()

print("\n" + "="*80)
print("CZĘŚĆ 1 ZAKOŃCZONA POWODZENIEM.")
print("Wszystkie dane i artefakty zostały zapisane w katalogach 'processed_data' i 'artifacts'.")
print("TERAZ NALEŻY ZRESTARTOWAĆ KERNEL PRZED URUCHOMIENIEM CZĘŚCI 2.")
print("="*80)

Wszystkie przetworzone zbiory danych zostały zapisane.

CZĘŚĆ 1 ZAKOŃCZONA POWODZENIEM.
Wszystkie dane i artefakty zostały zapisane w katalogach 'processed_data' i 'artifacts'.
TERAZ NALEŻY ZRESTARTOWAĆ KERNEL PRZED URUCHOMIENIEM CZĘŚCI 2.


# Część 2: Budowa, Trening i Inferencia Modelu
Witaj w nowej sesji kernela! Ta część wczyta przetworzone dane i artefakty, a następnie zdefiniuje, wytrenuje i wykorzysta model sieci neuronowej do predykcji lokalizacji z uwzględnieniem spójności hierarchicznej.

In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import gc
import scipy.sparse
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Ponowna definicja stałych
ARTIFACTS_DIR = 'artifacts'
PROCESSED_DATA_DIR = 'processed_data'
RANDOM_STATE = 42
MAX_TEXT_FEATURES = 10000 
MODEL_PATH = os.path.join(ARTIFACTS_DIR, 'best_location_model.keras')

## Wczytywanie Przetworzonych Danych i Artefaktów
Wczytujemy wszystkie niezbędne pliki wygenerowane w Części 1.

In [5]:
# KOMÓRKA 8 (Kod) - POPRAWIONA WERSJA Z MAPOWANIEM

# Wczytanie artefaktów
print("Wczytywanie artefaktów...")
hierarchy_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
id_to_name = joblib.load(os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))
scaler = joblib.load(os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))
vectorizer = joblib.load(os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))
print("Artefakty wczytane.")

# Wczytanie przetworzonych danych
print("Wczytywanie przetworzonych danych...")
X_train_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'))
X_val_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'))
X_train_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'))
X_val_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'))
X_train_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'))
X_val_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'))
y_train_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, 'y_train.csv'))
y_val_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, 'y_val.csv'))
y_train_orig = y_train_df.values.ravel()
y_val_orig = y_val_df.values.ravel()
print("Dane wczytane.")

# --- KLUCZOWA ZMIANA: MAPOWANIE ETYKIET ---
print("Tworzenie mapowania dla etykiet docelowych...")
# 1. Znajdź wszystkie unikalne, oryginalne ID lokalizacji
unique_targets = np.unique(np.concatenate([y_train_orig, y_val_orig]))
# 2. Stwórz mapę: oryginalne ID -> indeks od 0 do N-1
target_map = {val: i for i, val in enumerate(unique_targets)}
# 3. Stwórz mapę odwrotną: indeks od 0 do N-1 -> oryginalne ID
inverse_target_map = {i: val for val, i in target_map.items()}

# 4. Zapisz mapy jako artefakty do późniejszego użycia
joblib.dump(target_map, os.path.join(ARTIFACTS_DIR, 'target_map.joblib'))
joblib.dump(inverse_target_map, os.path.join(ARTIFACTS_DIR, 'inverse_target_map.joblib'))
print("Mapy etykiet zapisane.")

# 5. Zastosuj mapowanie do y_train i y_val
y_train = np.array([target_map[val] for val in y_train_orig])
y_val = np.array([target_map[val] for val in y_val_orig])
# --- KONIEC ZMIANY ---

# Obliczenie kluczowych parametrów
num_classes = len(unique_targets) # Poprawna liczba klas to liczba unikalnych ID
num_cities = int(np.concatenate([X_train_city, X_val_city]).max() + 1)

# Wyświetlanie kształtów
print("\n--- Kształty wczytanych danych ---")
print(f"X_train_text: {X_train_text.shape}")
print(f"X_train_num:  {X_train_num.shape}")
print(f"X_train_city: {X_train_city.shape}")
print(f"y_train (zmapowane): {y_train.shape}")
print(f"Num classes:  {num_classes}")
print(f"Num cities:   {num_cities}")

Wczytywanie artefaktów...
Artefakty wczytane.
Wczytywanie przetworzonych danych...
Dane wczytane.
Tworzenie mapowania dla etykiet docelowych...
Mapy etykiet zapisane.

--- Kształty wczytanych danych ---
X_train_text: (887572, 10000)
X_train_num:  (887572, 2)
X_train_city: (887572,)
y_train (zmapowane): (887572,)
Num classes:  41091
Num cities:   63004


## Definicja Architektury Modelu (Multi-Input, Single-Output)
Budujemy model Keras z trzema oddzielnymi wejściami (tekst, cechy numeryczne, miasto), które są łączone i przetwarzane przez wspólne warstwy gęste.

In [6]:
# Wejścia
input_text = Input(shape=(MAX_TEXT_FEATURES,), sparse=True, name='input_text')
input_num = Input(shape=(X_train_num.shape[1],), name='input_num')
input_city = Input(shape=(1,), name='input_city')

# Ścieżka Tekstowa
text_branch = Dense(128, activation='relu')(input_text)
text_branch = Dropout(0.5)(text_branch)

# Ścieżka Numeryczna
num_branch = Dense(32, activation='relu')(input_num)

# Ścieżka Miasta
city_branch = Embedding(input_dim=num_cities, output_dim=10, name='city_embedding')(input_city)
city_branch = Flatten()(city_branch)

# Połączenie
concatenated = Concatenate()([text_branch, num_branch, city_branch])

# Część Głęboka
deep_branch = Dense(256, activation='relu')(concatenated)
deep_branch = Dropout(0.5)(deep_branch)

# Wyjście
output_location = Dense(num_classes, activation='softmax', name='output_location')
output = output_location(deep_branch)

# Model
model = Model(inputs=[input_text, input_num, input_city], outputs=output)

# Kompilacja
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_text (InputLayer)       │ (None, 10000)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_city (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 128)               │       1,280,128 │ input_text[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_num (InputLayer)        │ (None, 2)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_embedding (Embedding)    │ (None, 1, 10)             │         630,040 │ input_city[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 128)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 32)                │              96 │ input_num[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_1 (Flatten)           │ (None, 10)                │               0 │ city_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 170)               │               0 │ dropout_2[0][0],           │
│                               │                           │                 │ dense_4[0][0],             │
│                               │                           │                 │ flatten_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_5 (Dense)               │ (None, 256)               │          43,776 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_3 (Dropout)           │ (None, 256)               │               0 │ dense_5[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_location (Dense)       │ (None, 41091)             │      10,560,387 │ dropout_3[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 12,514,427 (47.74 MB)

 Trainable params: 12,514,427 (47.74 MB)

 Non-trainable params: 0 (0.00 B)

## Trening Modelu
Uruchamiamy proces treningu z wykorzystaniem `EarlyStopping` w celu uniknięcia przeuczenia oraz `ModelCheckpoint` do zapisania najlepszej wersji modelu.

In [7]:
# Callbacks
model_checkpoint = ModelCheckpoint(
    MODEL_PATH, 
    monitor='val_loss', 
    save_best_only=True,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# Przygotowanie danych wejściowych jako słownik
X_train_dict = {
    'input_text': X_train_text,
    'input_num': X_train_num,
    'input_city': X_train_city
}

X_val_dict = {
    'input_text': X_val_text,
    'input_num': X_val_num,
    'input_city': X_val_city
}

# Trening
print("\nRozpoczynanie treningu modelu...")
history = model.fit(
    X_train_dict, 
    y_train,
    validation_data=(X_val_dict, y_val),
    epochs=20, 
    batch_size=64, 
    callbacks=[model_checkpoint, early_stopping]
)

print("\nTrening zakończony.")


Rozpoczynanie treningu modelu...
Epoch 1/20
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.1863 - loss: 6.5114
Epoch 1: val_loss improved from inf to 3.94440, saving model to artifacts\best_location_model.keras
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 1089s 78ms/step - accuracy: 0.1863 - loss: 6.5113 - val_accuracy: 0.3990 - val_loss: 3.9444
Epoch 2/20
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.3846 - loss: 4.0067
Epoch 2: val_loss improved from 3.94440 to 3.46167, saving model to artifacts\best_location_model.keras
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 1132s 82ms/step - accuracy: 0.3846 - loss: 4.0067 - val_accuracy: 0.4349 - val_loss: 3.4617
Epoch 3/20
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.4110 - loss: 3.5809
Epoch 3: val_loss improved from 3.46167 to 3.29005, saving model to artifacts\best_location_model.keras
13869/13869 ━━━━━━━━━━━━━━━━━━━━ 1194s 86ms/step - accuracy: 0.4110 - loss: 3.5809 - val_accuracy: 0.4492 - val_loss: 3.2901
Epoch 4/20

## Przygotowanie do Predykcji Hierarchicznej
Definiujemy funkcje pomocnicze, które zaimplementują kluczową logikę biznesową: odtworzenie pełnej ścieżki lokalizacji i weryfikację jej zgodności z miastem wejściowym.

In [8]:
# KOMÓRKA 11 (Kod) - POPRAWIONA WERSJA

def get_full_path(location_id, hierarchy_map, id_to_name):
    """Iteracyjnie odtwarza pełną ścieżkę ID lokalizacji od dołu do góry."""
    path_ids = []
    current_id = location_id
    for _ in range(10): 
        if pd.notna(current_id) and current_id in id_to_name:
            path_ids.append(int(current_id))
            current_id = hierarchy_map.get(current_id)
        else:
            break
    return path_ids

# --- KLUCZOWA ZMIANA: Aktualizacja funkcji, by używała mapy odwrotnej ---
def predict_with_hierarchy(model, inputs_dict, hierarchy_map, id_to_name, inverse_target_map, top_k=5):
    """Wykonuje predykcję, odmapowuje indeksy i weryfikuje je z hierarchią."""
    # Wczytujemy mapę odwrotną, jeśli jej nie ma
    if 'inverse_target_map' not in globals():
        globals()['inverse_target_map'] = joblib.load(os.path.join(ARTIFACTS_DIR, 'inverse_target_map.joblib'))

    predictions = model.predict(inputs_dict)
    # argsort zwraca ZMAPOWANE indeksy [0, N-1]
    top_k_mapped_indices = np.argsort(predictions[0])[::-1][:top_k]
    
    input_city_id = inputs_dict['input_city'][0]
    
    for mapped_idx in top_k_mapped_indices:
        # Odmapuj indeks na oryginalne, duże ID lokalizacji
        candidate_id = inverse_target_map.get(mapped_idx)
        if candidate_id is None:
            continue # Pomiń, jeśli z jakiegoś powodu indeks jest niepoprawny
            
        path = get_full_path(candidate_id, hierarchy_map, id_to_name)
        
        if input_city_id in path:
            return candidate_id # Zwracamy ORYGINALNE ID

    # Fallback: jeśli żaden nie pasuje, zwróć pierwszego kandydata
    top_candidate_id = inverse_target_map.get(top_k_mapped_indices[0])
    return top_candidate_id
# --- KONIEC ZMIANY ---

print("Funkcje pomocnicze do predykcji hierarchicznej zdefiniowane.")

Funkcje pomocnicze do predykcji hierarchicznej zdefiniowane.


## Wykonanie i Ocena Predykcji na Zbiorze Walidacyjnym
Używamy zdefiniowanych funkcji do przeprowadzenia predykcji na zbiorze walidacyjnym i obliczamy ogólną dokładność naszego podejścia.

In [9]:
# KOMÓRKA 12 (Kod) - POPRAWIONA WERSJA

# Wczytaj najlepszy zapisany model
print(f"Wczytywanie najlepszego modelu z: {MODEL_PATH}")
best_model = tf.keras.models.load_model(MODEL_PATH)

# Wczytujemy mapę odwrotną
inverse_target_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'inverse_target_map.joblib'))

# Wybierzmy podzbiór danych walidacyjnych do szybkiej ewaluacji
num_samples = 1000
indices = np.random.choice(range(len(y_val)), num_samples, replace=False)

predictions = []
true_labels = []

print(f"Przeprowadzanie predykcji na {num_samples} próbkach...")
for i in indices:
    input_sample = {
        'input_text': X_val_text[i],
        'input_num': np.expand_dims(X_val_num[i], axis=0),
        'input_city': np.expand_dims(X_val_city[i], axis=0)
    }
    
    # Przekazujemy mapę do funkcji
    predicted_id = predict_with_hierarchy(best_model, input_sample, hierarchy_map, id_to_name, inverse_target_map)
    predictions.append(predicted_id)
    
    # Odmapowujemy prawdziwą etykietę z powrotem do oryginalnego ID
    true_mapped_id = y_val[i]
    true_original_id = inverse_target_map[true_mapped_id]
    true_labels.append(true_original_id)

# Tworzenie DataFrame z wynikami
df_results = pd.DataFrame({
    'Original_Location_ID': true_labels,
    'Predicted_Location_ID': predictions
})

# Dodawanie nazw dla czytelności
df_results['Original_Location_Name'] = df_results['Original_Location_ID'].apply(lambda x: id_to_name.get(x, 'Nie znaleziono'))
df_results['Predicted_Location_Name'] = df_results['Predicted_Location_ID'].apply(lambda x: id_to_name.get(x, 'Nie znaleziono'))

# Sprawdzanie poprawności
df_results['Is_Correct'] = (df_results['Original_Location_ID'] == df_results['Predicted_Location_ID'])

# Obliczenie i wyświetlenie dokładności
accuracy = df_results['Is_Correct'].mean()
print(f"\nDokładność predykcji na {num_samples} próbkach walidacyjnych: {accuracy:.2%}")

Wczytywanie najlepszego modelu z: artifacts\best_location_model.keras
Przeprowadzanie predykcji na 1000 próbkach...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

## Wizualizacja Wyników Predykcji
Wyświetlamy losowe przykłady predykcji, kolorując wiersze na zielono (poprawne) i czerwono (błędne), aby wizualnie ocenić jakość działania modelu.

In [10]:
def highlight_correct(s):
    """Koloruje tło wiersza na podstawie wartości w kolumnie 'Is_Correct'."""
    return ['background-color: #d4edda' if s.Is_Correct else 'background-color: #f8d7da'] * len(s)

# Wyświetlenie 20 losowych wierszy ze stylizacją
styled_results = df_results.sample(min(20, len(df_results)), random_state=RANDOM_STATE).style.apply(highlight_correct, axis=1)

print("\n--- Losowe wyniki predykcji (Zielony = Poprawna, Czerwony = Błędna) ---")
display(styled_results)


--- Losowe wyniki predykcji (Zielony = Poprawna, Czerwony = Błędna) ---


,Original_Location_ID,Predicted_Location_ID,Original_Location_Name,Predicted_Location_Name,Is_Correct
521,333,333,Bielsko-biała,Bielsko-biała,True
737,384,384,Bytom,Bytom,True
740,17796,17796,Kowale,Kowale,True
660,267837,70029,Osiedle kalinowe,Bieńczyce,False
411,68566,68566,Rataje,Rataje,True
678,362061,69530,Józefa,Górna,False
626,105391,100392,Prokocim,Krowodrza,False
513,105466,105466,Śródmieście,Śródmieście,True
859,65196,69149,Teofilów,Bałuty,False
136,531655,531655,Katowicka,Katowicka,True
